# From IMDB Dataset
 * [IMDb Datasets - Homepage](https://www.imdb.com/interfaces/)
 * [IMDb Datasets - Linkpage](https://datasets.imdbws.com)
## Productors 

In [8]:
# title.basics 
#   ID Film / Movie / Title / Year / Genres 

# title.ratings
#   ID Film / Note 

# title.crew
#   ID Film / ID Director 

# name.basics
#   ID Director / Name 

import pandas as pd
import numpy as np
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [9]:
link = 'https://datasets.imdbws.com/title.basics.tsv.gz'
title_basics = pd.read_table(link, on_bad_lines='skip')
title_basics = title_basics[title_basics['titleType'] == 'movie']
title_basics = title_basics.drop(columns=['titleType','primaryTitle','isAdult', 'endYear','runtimeMinutes'])
title_basics = title_basics[title_basics['originalTitle'] != r'\N']
title_basics = title_basics[title_basics['startYear'] != r'\N']
title_basics = title_basics[title_basics['genres'] != r'\N']

/var/folders/vh/l2wksfvx397_s94nz191rypm0000gn/T/ipykernel_48923/2554974781.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  title_basics = pd.read_table(link, on_bad_lines='skip')


In [10]:
link = 'https://datasets.imdbws.com/title.ratings.tsv.gz'
title_ratings = pd.read_table(link, on_bad_lines='skip')
title_basics_ratings = pd.merge(title_basics, title_ratings, left_on='tconst', right_on='tconst')

In [11]:
link = 'https://datasets.imdbws.com/title.crew.tsv.gz'
title_crew = pd.read_table(link, on_bad_lines='skip')
title_crew = title_crew.drop(columns=['writers'])
title_basics_ratings_crew = pd.merge(title_basics_ratings, title_crew, left_on='tconst', right_on='tconst')
title_basics_ratings_crew["directors"] = [ str(x).replace(' ', '') for x in title_basics_ratings_crew["directors"][:] ] 
title_basics_ratings_crew["directors"] = [ x.split(",") for x in title_basics_ratings_crew["directors"][:] ]
title_basics_ratings_crew = title_basics_ratings_crew.explode('directors').reset_index()

In [12]:
link = 'https://datasets.imdbws.com/name.basics.tsv.gz'
name_basics = pd.read_table(link, on_bad_lines='skip')
name_basics = name_basics.drop(columns=['birthYear','deathYear','primaryProfession','knownForTitles'])
title_basics_ratings_crew_name = pd.merge(title_basics_ratings_crew, name_basics, left_on='directors', right_on='nconst')
title_basics_ratings_crew_name = title_basics_ratings_crew_name.drop(columns=['index','directors','nconst'])
title_basics_ratings_crew_name["startYear"] = pd.to_numeric(title_basics_ratings_crew_name["startYear"]).astype(int)

In [13]:
import urllib.parse
# https://www.imdb.com/search/title/?title=TOUT+S%27EST+BIEN+PASSE&release_date=2015-01-01,2022-12-31

url_base = 'https://www.imdb.com/search/title/?title='

title_basics_ratings_crew_name['imdb_search'] = ''

for i in range( len(title_basics_ratings_crew_name) ):

    title = urllib.parse.quote(title_basics_ratings_crew_name.iloc[i]['originalTitle']).replace('%20', '+')
    date = str(title_basics_ratings_crew_name.iloc[i]['startYear'])

    title_basics_ratings_crew_name.imdb_search[i] = f"{url_base}{title}&release_date={date}-01-01,{date}-12-31"
    
title_basics_ratings_crew_name.head()

/var/folders/vh/l2wksfvx397_s94nz191rypm0000gn/T/ipykernel_48923/1728146170.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  title_basics_ratings_crew_name.imdb_search[i] = f"{url_base}{title}&release_date={date}-01-01,{date}-12-31"


,tconst,originalTitle,startYear,genres,averageRating,numVotes,primaryName,imdb_search
0,tt0000009,Miss Jerry,1894,Romance,5.2,200,Alexander Black,https://www.imdb.com/search/title/?title=Miss+...
1,tt0000574,The Story of the Kelly Gang,1906,"Action,Adventure,Biography",6.0,784,Charles Tait,https://www.imdb.com/search/title/?title=The+S...
2,tt0000591,L'enfant prodigue,1907,Drama,4.0,19,Michel Carré,https://www.imdb.com/search/title/?title=L%27e...
3,tt0000615,Robbery Under Arms,1907,Drama,4.1,23,Charles MacMahon,https://www.imdb.com/search/title/?title=Robbe...
4,tt0000630,Amleto,1908,Drama,2.9,25,Mario Caserini,https://www.imdb.com/search/title/?title=Amlet...


In [14]:
title_basics_ratings_crew_name.to_csv(r'src/dataset_without_resume.csv', index = False, header=True)